# Aggregation EIE dataset for school level and matching it with school finance

In [45]:
import pandas as pd

In [46]:
# ZNO==EIE
df_preprocessed = pd.read_csv("zno_preprocessed.csv")

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (19,28,37,46,55,64,73,81,90,99,106,107,108,109,111,112,113,114) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [47]:
fin_datasets = []
year=2018
for name in ['ZZSO-2018.xlsx','ZZSO-2019.xlsx','ZZSO-2020.xlsx']:
    dataset = pd.read_excel('fin_data/'+name)
    dataset.columns = dataset.loc[0].values
    dataset = dataset.iloc[1:]
    dataset = dataset[(dataset['Ступінь закладу ( I, I-II, I-III)']!=0)&(dataset['Ступінь закладу ( I, I-II, I-III)'].str.contains('III'))].reset_index(drop=True)
    dataset['year'] = year
    year = year+1
    fin_datasets.append(dataset)

fin_datasets = pd.concat(fin_datasets).reset_index(drop=True)

In [48]:
fin_datasets.head()

,№ Області,Область,Тип територіальної одиниці,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,ЄДРПОУ,Примітка,"Питома вага непедагогічного персоналу у загальній кількості працівників, %","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %","Фактична наповнюваність класів, учнів на клас","Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)"
0,4,Дніпропетровська,Район,1963,П'ятихатський район,"Троїцький навчально- виховний комплекс ""загаль...",I-III,2766,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,Донецька,Місто,2370,м. Маріуполь,"Комунальний заклад ""Маріупольська загальноосві...",I-III,0,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,Донецька,Район,2468,Великоновосілківський район,Зеленопільська загальноосвітня школа І-ІІІ сту...,I-III,830,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,Донецька,Район,2536,Волноваський район,Зачатівська загальноосвітня школа№2 Волноваськ...,I-III,0,0,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Запорізька,Район,4365,Токмацький р-н,Ударнецька ЗОШ,I-III,0,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# matching key (get from match_schools_{year} folders)
match_datasets = []
for name in ['match_schools_2018/finance_schools_matched_2018.csv',
             'match_schools_2019/finance_schools_matched_2019.csv',
             'match_schools_2020/finance_schools_matched_2020.csv']:
    dataset = pd.read_csv(name)
    match_datasets.append(dataset)

match_df = pd.concat(match_datasets).drop_duplicates().reset_index(drop=True)
match_df

,eoname,schoolname
0,Нововолинська гімназія Нововолинської міської ...,Нововолинська гімназія Нововолинської міської ...
1,Шаргородська районна гімназія,Шаргородська районна гімназія
2,"Комунальний заклад ""Вінницький технічний ліцей""","Комунальний заклад ""Вінницький технічний ліцей"""
3,Комунальний заклад загальної середньої освіти ...,Комунальний заклад загальної середньої освіти ...
4,Марганецька загальноосвітня школа І-ІІІ ступен...,Марганецька загальноосвітня школа І-ІІІ ступен...
...,...,...
13621,Комунальний заклад Липоводолинська спеціалізов...,Липоводолинська спеціалізована школа І-ІІІ сту...
13622,Опорний навчальний заклад «Купільська загально...,Купільська загальноосвітня школа І-ІІІ ступені...
13623,"Старокривинський навчально-виховний комплекс ""...","Старокривинський навчально-виховний комплекс ""..."
13624,Чернівецька загальноосвітня школа І-ІІІ ступен...,загальноосвітня школа I-III ступенів №14_x000D...


In [50]:
fin_datasets['eoname'] = fin_datasets['Найменування закладу'].map(
    lambda x: match_df[match_df['schoolname']==x]['eoname'].values[0] if x in match_df['schoolname'].unique() else None
)
fin_datasets.head()

,№ Області,Область,Тип територіальної одиниці,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,Примітка,"Питома вага непедагогічного персоналу у загальній кількості працівників, %","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %","Фактична наповнюваність класів, учнів на клас","Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",eoname
0,4,Дніпропетровська,Район,1963,П'ятихатський район,"Троїцький навчально- виховний комплекс ""загаль...",I-III,2766,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Троїцький навчально-виховний комплекс ""загальн..."
1,5,Донецька,Місто,2370,м. Маріуполь,"Комунальний заклад ""Маріупольська загальноосві...",I-III,0,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,5,Донецька,Район,2468,Великоновосілківський район,Зеленопільська загальноосвітня школа І-ІІІ сту...,I-III,830,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,5,Донецька,Район,2536,Волноваський район,Зачатівська загальноосвітня школа№2 Волноваськ...,I-III,0,0,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,8,Запорізька,Район,4365,Токмацький р-н,Ударнецька ЗОШ,I-III,0,0,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [51]:
fin_datasets = fin_datasets.dropna(subset=['eoname']).reset_index(drop=True)

### Delete not relevant columns

In [52]:
fin_datasets = fin_datasets.drop(columns=['№ Області', 'Область', 'Тип територіальної одиниці', '№№','Загальна площа приміщень школи ( без господарських будівель) (кв.м.)',
'Назва тер. Одиниці', 'Найменування закладу', 'Тип школи: міська / сільська', 'Питома вага непедагогічного персоналу у загальній кількості працівників, %**',
       'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**',
'Фактична наповнюваність класів, учнів на клас**',
       'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**',
       'Видатки на 1 учня, 2018, тис. грн.**','Видатки на 1 учня, 2019, тис. грн.**',
       'Видатки на 1 клас, тис. грн.** 2019 рік (каса)', 'Опорна',
       'Код бюджету', 'Форма власності', 'Код ДІСО', 'ЄДРПОУ', 'Примітка',
        'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя',
       ])

In [53]:
df_preprocessed = df_preprocessed.drop(columns=['Unnamed: 0'])

In [86]:
orig_df = df_preprocessed.drop(['outid', 'birth', 'regname', 'areaname', 'tername',
                        'regtypename', 'tertypename', 'ukrptname', 'ukrptregname',
                        'ukrptareaname', 'ukrpttername'], axis=1)

In [87]:
orig_df = orig_df.drop(['histptname', 'histptregname',
                        'histptareaname', 'histpttername',
                        'mathptname', 'mathptregname',
                        'mathptareaname', 'mathpttername',
                        'physptname', 'physptregname',
                        'physptareaname', 'physpttername',
                        'chemptname', 'chemptregname',
                        'chemptareaname', 'chempttername',
                        'bioptname', 'bioptregname',
                        'bioptareaname', 'biopttername',
                        'geoptname', 'geoptregname',
                        'geoptareaname', 'geopttername',
                        'engptname', 'engptregname',
                        'engptareaname', 'engpttername',
                        'fraptname', 'fraptregname',
                        'fraptareaname', 'frapttername',
                        'deuptname', 'deuptregname',
                        'deuptareaname', 'deupttername',
                        'spaptname', 'spaptregname',
                        'spaptareaname', 'spapttername',], axis=1)

### Aggregation EIE dataset for school level

In [88]:
orig_df["sexM"] = (orig_df["sextypename"] == "чоловіча").astype(int)
orig_df["sexF"] = (orig_df["sextypename"] == "жіноча").astype(int)

In [89]:
orig_df["phys_mat"] = (orig_df["classprofilename"] == 'Фізико-математичний').astype(int)
orig_df["foreign_phyl"] = (orig_df["classprofilename"] == 'Іноземної філології').astype(int)
orig_df["universal"] = (orig_df["classprofilename"] == 'Універсальний').astype(int)
orig_df["ukr_phyl"] = (orig_df["classprofilename"] == 'Української філології').astype(int)
orig_df["technology"] = (orig_df["classprofilename"] == 'Технологічний').astype(int)
orig_df["bio_phys"] = (orig_df["classprofilename"] == 'Біолого-фізичний').astype(int)
orig_df["economy"] = (orig_df["classprofilename"] == 'Економічний').astype(int)
orig_df["history"] = (orig_df["classprofilename"] == 'Історичний').astype(int)
orig_df["sport"] = (orig_df["classprofilename"] == 'Спортивний').astype(int)
orig_df["math"] = (orig_df["classprofilename"] == 'Математичний').astype(int)
orig_df["law"] = (orig_df["classprofilename"] == 'Правовий').astype(int)
orig_df["other"] = (orig_df["classprofilename"] == 'Інший(багатопрофільність)').astype(int)
orig_df["martial_sport"] = (orig_df["classprofilename"] == 'Військово-спортивний').astype(int)
orig_df["philosophy"] = (orig_df["classprofilename"] == 'Філософський').astype(int)
orig_df["ecology"] = (orig_df["classprofilename"] == 'Екологічний').astype(int)
orig_df["arts"] = (orig_df["classprofilename"] == 'Художньо-естетичний').astype(int)
orig_df["geography"] = (orig_df["classprofilename"] == 'Географічний').astype(int)
orig_df["physics"] = (orig_df["classprofilename"] == 'Фізичний').astype(int)
orig_df["chem_tech"] = (orig_df["classprofilename"] == 'Хіміко-технологічний та агрохімічний').astype(int)
orig_df["biology"] = (orig_df["classprofilename"] == 'Біологічний').astype(int)
orig_df["phys_chem"] = (orig_df["classprofilename"] == 'Фізико-хімічний').astype(int)
orig_df["young_spec"] = (orig_df["classprofilename"] == 'Молодший спеціаліст').astype(int)
orig_df["qual_work"] = (orig_df["classprofilename"] == 'Кваліфікований робітник').astype(int)

In [90]:
orig_df["hung_lang"] = (orig_df["classlangname"] == 'угорська').astype(int)
orig_df["ukr_lang"] = (orig_df["classlangname"] == 'українська').astype(int)
orig_df["rus_lang"] = (orig_df["classlangname"] == 'російська').astype(int)
orig_df["roman_lang"] = (orig_df["classlangname"] == 'румунська').astype(int)
orig_df["pol_lang"] = (orig_df["classlangname"] == 'польська').astype(int)
orig_df["mold_lang"] = (orig_df["classlangname"] == 'молдовська').astype(int)
orig_df["other_lang"] = (orig_df["classlangname"] == 'інша').astype(int)

In [91]:
def change_teststatus(test):
    orig_df[f"{test}_accept"] = (orig_df[f"{test}teststatus"] == 'Зараховано').astype(int)
    orig_df[f"{test}_fail"] = (orig_df[f"{test}teststatus"] == 'Не подолав поріг').astype(int)
    orig_df[f"{test}_abscent"] = (orig_df[f"{test}teststatus"] == 'Не з’явився').astype(int)
    orig_df[f"{test}_annul"] = (orig_df[f"{test}teststatus"] == 'Анульовано').astype(int)
    orig_df[f"{test}_no_choice"] = (orig_df[f"{test}teststatus"] == 'Не обрано 100-200').astype(int)

In [92]:
change_teststatus("ukr")
change_teststatus("hist")
change_teststatus("math")
change_teststatus("phys")
change_teststatus("chem")
change_teststatus("bio")
change_teststatus("geo")
change_teststatus("eng")
change_teststatus("fra")
change_teststatus("deu")
change_teststatus("spa")

In [93]:
def change_testlang(test):
    orig_df[f"{test}_ukrlang"] = (orig_df[f"{test}lang"] == 'українська').astype(int)
    orig_df[f"{test}_ruslang"] = (orig_df[f"{test}lang"] == 'російська').astype(int)
    orig_df[f"{test}_moldlang"] = (orig_df[f"{test}lang"] == 'молдовська').astype(int)
    orig_df[f"{test}_hunglang"] = (orig_df[f"{test}lang"] == 'угорська').astype(int)
    orig_df[f"{test}_romlang"] = (orig_df[f"{test}lang"] == 'румунська').astype(int)
    orig_df[f"{test}_pollang"] = (orig_df[f"{test}lang"] == 'польська').astype(int)

In [94]:
change_testlang("hist")
change_testlang("math")
change_testlang("phys")
change_testlang("chem")
change_testlang("bio")
change_testlang("geo")

/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys
/Users/nazardrushchak/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

In [95]:
orig_df = orig_df.drop(['sextypename'], axis=1)

In [96]:
to_remove = ['classprofilename', 'classlangname', 'ukrteststatus', 'histteststatus',
                        'mathteststatus', 'physteststatus', 'chemteststatus', 'bioteststatus',
                        'geoteststatus', 'engteststatus', 'frateststatus', 'deuteststatus',
                        'spateststatus', 'mathlang', 'histlang', 'physlang', 'chemlang',
                        'biolang', 'geolang']
for feature in to_remove:
    orig_df = orig_df.drop([feature], axis=1)

In [97]:
orig_df["eng_dpa_standard"] = (orig_df["engdpalevel"] == 'стандарт(академічний)').astype(int)
orig_df["eng_dpa_prof"] = (orig_df["engdpalevel"] == 'профільний').astype(int)
orig_df["fra_dpa_standard"] = (orig_df["fradpalevel"] == 'стандарт(академічний)').astype(int)
orig_df["fra_dpa_prof"] = (orig_df["fradpalevel"] == 'профільний').astype(int)
orig_df["deu_dpa_standard"] = (orig_df["deudpalevel"] == 'стандарт(академічний)').astype(int)
orig_df["deu_dpa_prof"] = (orig_df["deudpalevel"] == 'профільний').astype(int)
orig_df["spa_dpa_standard"] = (orig_df["spadpalevel"] == 'стандарт(академічний)').astype(int)
orig_df["spa_dpa_prof"] = (orig_df["spadpalevel"] == 'профільний').astype(int)

In [98]:
orig_df = orig_df.drop(['engdpalevel', 'fradpalevel', 'deudpalevel', 'spadpalevel'], axis=1)

In [99]:
def q25(x):
    return x.quantile(0.25)

def q50(x):
    return x.quantile(0.5)

def q75(x):
    return x.quantile(0.75)

In [109]:
orig_df['ukrball100'] = pd.to_numeric(orig_df['ukrball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['histball100'] = pd.to_numeric(orig_df['histball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['mathball100'] = pd.to_numeric(orig_df['mathball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['physball100'] = pd.to_numeric(orig_df['physball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['chemball100'] = pd.to_numeric(orig_df['chemball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['bioball100'] = pd.to_numeric(orig_df['bioball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['geoball100'] = pd.to_numeric(orig_df['geoball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['engball100'] = pd.to_numeric(orig_df['engball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['fraball100'] = pd.to_numeric(orig_df['fraball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['deuball100'] = pd.to_numeric(orig_df['deuball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))
orig_df['spaball100'] = pd.to_numeric(orig_df['spaball100'].map(lambda x: str(x).replace(',','.') if pd.notna(x) else None))

In [110]:
grouped = orig_df.groupby(["eoname", "year"],dropna=True).agg({
    'sexM': 'sum',
    'sexF': 'sum',
    
    'phys_mat': 'sum',
    'foreign_phyl': 'sum',
    'universal': 'sum',
    'ukr_phyl': 'sum',
    'technology': 'sum',
    'bio_phys': 'sum',
    'economy': 'sum',
    'history': 'sum',
    'sport': 'sum',
    'math': 'sum',
    'law': 'sum',
    'other': 'sum',
    'martial_sport': 'sum',
    'philosophy': 'sum',
    'ecology': 'sum',
    'arts': 'sum',
    'geography': 'sum',
    'physics': 'sum',
    'chem_tech': 'sum',
    'biology': 'sum',
    'phys_chem': 'sum',
    'young_spec': 'sum',
    'qual_work': 'sum',
    
    'hung_lang': 'max',
    'ukr_lang': 'max',
    'rus_lang': 'max',
    'roman_lang': 'max',
    'pol_lang': 'max',
    'mold_lang': 'max',
    'other_lang': 'max',
    
    'eotypename': 'min',
    'eoregname': 'min',
    'eoareaname': 'min',
    'eotername': 'min',
    'eoparent': 'min',
    
    'ukrtest': 'count',
    'ukr_accept': 'sum',
    'ukr_fail': 'sum',
    'ukr_abscent': 'sum',
    'ukr_annul': 'sum',
    'ukr_no_choice': 'sum',
    'ukrball100': ['min', 'max', q25, q50, q75, 'mean'],
    'ukrball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'histtest': 'count',
    'hist_ukrlang': 'sum',
    'hist_ruslang': 'sum',
    'hist_moldlang': 'sum',
    'hist_hunglang': 'sum',
    'hist_romlang': 'sum',
    'hist_pollang': 'sum',
    'hist_accept': 'sum',
    'hist_fail': 'sum',
    'hist_abscent': 'sum',
    'hist_annul': 'sum',
    'hist_no_choice': 'sum',
    'histball100': ['min', 'max', q25, q50, q75, 'mean'],
    'histball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'mathtest': 'count',
    'math_ukrlang': 'sum',
    'math_ruslang': 'sum',
    'math_moldlang': 'sum',
    'math_hunglang': 'sum',
    'math_romlang': 'sum',
    'math_pollang': 'sum',
    'math_accept': 'sum',
    'math_fail': 'sum',
    'math_abscent': 'sum',
    'math_annul': 'sum',
    'math_no_choice': 'sum',
    'mathball100': ['min', 'max', q25, q50, q75, 'mean'],
    'mathball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'phystest': 'count',
    'phys_ukrlang': 'sum',
    'phys_ruslang': 'sum',
    'phys_moldlang': 'sum',
    'phys_hunglang': 'sum',
    'phys_romlang': 'sum',
    'phys_pollang': 'sum',
    'phys_accept': 'sum',
    'phys_fail': 'sum',
    'phys_abscent': 'sum',
    'phys_annul': 'sum',
    'phys_no_choice': 'sum',
    'physball100': ['min', 'max', q25, q50, q75, 'mean'],
    'physball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'chemtest': 'count',
    'chem_ukrlang': 'sum',
    'chem_ruslang': 'sum',
    'chem_moldlang': 'sum',
    'chem_hunglang': 'sum',
    'chem_romlang': 'sum',
    'chem_pollang': 'sum',
    'chem_accept': 'sum',
    'chem_fail': 'sum',
    'chem_abscent': 'sum',
    'chem_annul': 'sum',
    'chem_no_choice': 'sum',
    'chemball100': ['min', 'max', q25, q50, q75, 'mean'],
    'chemball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'histtest': 'count',
    'hist_ukrlang': 'sum',
    'hist_ruslang': 'sum',
    'hist_moldlang': 'sum',
    'hist_hunglang': 'sum',
    'hist_romlang': 'sum',
    'hist_pollang': 'sum',
    'hist_accept': 'sum',
    'hist_fail': 'sum',
    'hist_abscent': 'sum',
    'hist_annul': 'sum',
    'hist_no_choice': 'sum',
    'histball100': ['min', 'max', q25, q50, q75, 'mean'],
    'histball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'biotest': 'count',
    'bio_ukrlang': 'sum',
    'bio_ruslang': 'sum',
    'bio_moldlang': 'sum',
    'bio_hunglang': 'sum',
    'bio_romlang': 'sum',
    'bio_pollang': 'sum',
    'bio_accept': 'sum',
    'bio_fail': 'sum',
    'bio_abscent': 'sum',
    'bio_annul': 'sum',
    'bio_no_choice': 'sum',
    'bioball100': ['min', 'max', q25, q50, q75, 'mean'],
    'bioball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'geotest': 'count',
    'geo_ukrlang': 'sum',
    'geo_ruslang': 'sum',
    'geo_moldlang': 'sum',
    'geo_hunglang': 'sum',
    'geo_romlang': 'sum',
    'geo_pollang': 'sum',
    'geo_accept': 'sum',
    'geo_fail': 'sum',
    'geo_abscent': 'sum',
    'geo_annul': 'sum',
    'geo_no_choice': 'sum',
    'geoball100': ['min', 'max', q25, q50, q75, 'mean'],
    'geoball': ['min', 'max', q25, q50, q75, 'mean'],

    'engtest': 'count',
    'eng_accept': 'sum',
    'eng_fail': 'sum',
    'eng_abscent': 'sum',
    'eng_annul': 'sum',
    'eng_no_choice': 'sum',
    'engball100': ['min', 'max', q25, q50, q75, 'mean'],
    'eng_dpa_standard': 'sum',
    'eng_dpa_prof': 'sum',
    'engball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'fratest': 'count',
    'fra_accept': 'sum',
    'fra_fail': 'sum',
    'fra_abscent': 'sum',
    'fra_annul': 'sum',
    'fra_no_choice': 'sum',
    'fraball100': ['min', 'max', q25, q50, q75, 'mean'],
    'fra_dpa_standard': 'sum',
    'fra_dpa_prof': 'sum',
    'fraball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'deutest': 'count',
    'deu_accept': 'sum',
    'deu_fail': 'sum',
    'deu_abscent': 'sum',
    'deu_annul': 'sum',
    'deu_no_choice': 'sum',
    'deuball100': ['min', 'max', q25, q50, q75, 'mean'],
    'deu_dpa_standard': 'sum',
    'deu_dpa_prof': 'sum',
    'deuball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'spatest': 'count',
    'spa_accept': 'sum',
    'spa_fail': 'sum',
    'spa_abscent': 'sum',
    'spa_annul': 'sum',
    'spa_no_choice': 'sum',
    'spaball100': ['min', 'max', q25, q50, q75, 'mean'],
    'spa_dpa_standard': 'sum',
    'spa_dpa_prof': 'sum',
    'spaball': ['min', 'max', q25, q50, q75, 'mean'],
    
    'ukradaptscale': 'first',
})
print("SUCCESS")

SUCCESS


In [111]:
grouped = grouped.reset_index()

In [112]:
grouped.columns = [col[0] if col[1] in ['sum','','first'] else f"{col[0]}_{col[1]}" for col in grouped.columns]
grouped.head()

,eoname,year,sexM,sexF,phys_mat,foreign_phyl,universal,ukr_phyl,technology,bio_phys,...,spaball100_mean,spa_dpa_standard,spa_dpa_prof,spaball_min,spaball_max,spaball_q25,spaball_q50,spaball_q75,spaball_mean,ukradaptscale
0,""" Селищенський заклад загальної середньої осві...",2021,5,5,0,0,0,10,0,0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,"""Баламутівський заклад загальної середньої осв...",2021,6,3,0,0,0,9,0,0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,"""Великолазівський ліцей"" Баранинської сільсько...",2021,15,22,0,0,0,0,0,0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,"""Вінницько-Хутірський навчально-виховний компл...",2021,7,2,0,0,0,9,0,0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,"""Демино-Олександрівський ліцей Троїцької селищ...",2019,3,5,0,0,8,0,0,0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


### Merge datasets

In [115]:
school_dataset = pd.merge(grouped, fin_datasets, on=['eoname','year'], how='inner')
school_dataset

,eoname,year,sexM,sexF,phys_mat,foreign_phyl,universal,ukr_phyl,technology,bio_phys,...,"Разом кількість учнів, та дітей, які здобувають дошкільну освіту","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Питома вага непедагогічного персоналу у загальній кількості працівників, %","Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %","Фактична наповнюваність класів, учнів на клас","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)"
0,"""Демино-Олександрівський ліцей Троїцької селищ...",2019,3,5,0,0,8,0,0,0,...,59,3468.816,4066.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""Демино-Олександрівський ліцей Троїцької селищ...",2020,6,3,0,0,9,0,0,0,...,NaN,NaN,NaN,0.285714,0.133333,7.75,3620.1,4862.1,58.38871,452.5125
2,"""Лантратівський ліцей Троїцької селищної ради ...",2019,8,4,0,0,12,0,0,0,...,95,4170.501,4652.336,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Лантратівський ліцей Троїцької селищної ради ...",2020,2,2,0,0,4,0,0,0,...,NaN,NaN,NaN,0.346154,0.235294,8.727273,4298.7,5290.8,44.778125,390.790909
4,"""Навчально-виховний комплекс ""Школа І-ІІ ступе...",2019,23,21,0,20,0,0,0,0,...,778,18463.07,28107.648,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18632,"опорний заклад освіти ""Красилівський академічн...",2020,7,8,0,0,0,0,0,0,...,NaN,NaN,NaN,0.477273,0,8.214286,6302.083323,6429.120117,54.800725,450.148809
18633,опорний заклад освіти «Ставрівський навчально-...,2019,6,5,0,0,0,11,0,0,...,424,10841.395,12277.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18634,"опорний навчальний заклад ""Апостолівська загал...",2019,11,11,0,0,0,13,0,0,...,844,19541,21160.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18635,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,2019,20,20,0,0,0,0,0,0,...,1011,25717.609,31263.175,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Save dataset to csv

In [116]:
school_dataset.to_csv('school_dataset_sample.csv',index=False)